In [ ]:
# training models for najalee/sum-it

In [ ]:
pip install transformers[torch]

In [ ]:
pip install torch


In [ ]:
pip install transformers datasets evaluate rouge_score

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import os

current_path = os.environ['PATH']

git_cmd_path = r"C:\Program Files\Git\cmd"
git_bin_path = r"C:\Program Files\Git\bin"

# add paths to current path
paths_to_add = [git_cmd_path, git_bin_path]
for path in paths_to_add:
    if path not in current_path:
        current_path = path + ";" + current_path 
        print(f"Added: {path}")

os.environ['PATH'] = current_path


In [ ]:
os.environ['PATH']

In [ ]:
!git clone https://github.com/najalee/nlp-project.git

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset

ds = load_dataset("abisee/cnn_dailymail", "3.0.0")

In [ ]:
ds = ds["train"].train_test_split(test_size=0.2)

In [ ]:
ds

In [ ]:
# now we preprocess the loaded datatset
from transformers import AutoTokenizer

In [ ]:
# load the google t5 tokenizer

loadedTokenizer = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(loadedTokenizer)

In [ ]:
# tell T5 that this is a summzarization task
prefix = "summarize: "

def preprocess_function(artSum):
    inputs = [prefix + doc for doc in artSum["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)


    labels = tokenizer(text_target=artSum["highlights"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_ds = {}
tokenized_ds["train"] = ds["train"].shuffle(seed=42).select(range(20000)).map(preprocess_function, batched=True) # create a small dataset so training doesnt take 500 years
tokenized_ds["test"] = ds["test"].shuffle(seed=42).select(range(1000)).map(preprocess_function, batched=True)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=loadedTokenizer)

In [ ]:
# evaluate w/ rouge score
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}
    

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(loadedTokenizer)

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))



In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="sum-it_model",
    eval_strategy="no",
    learning_rate=2e-5,
    #per_device_train_batch_size=16,
    #per_device_eval_batch_size=16,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=False,
    fp16=True, #change to bf16=True for XPU
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# training_args = Seq2SeqTrainingArguments(
#     output_dir="my_awesome_billsum_model",
#     #eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     weight_decay=0.01,
#     save_total_limit=3,
#     num_train_epochs=4,
#     predict_with_generate=True,
#     fp16=True, #change to bf16=True for XPU
#     push_to_hub=False,
#     #dataloader_num_workers=4,  # or 8, depending on your CPU
# )

# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_ds["train"],
#     eval_dataset=tokenized_ds["test"],
#     processing_class=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )

# trainer.train()

In [ ]:
trainer.push_to_hub()